In [ ]:
##################################    ECPY BOILERPLATE CODE    ##################################

# !pip install ecpy
from random import randint, seed
from ecpy.curves import Curve
from Crypto.Hash import SHA3_256
from Crypto import Random   # a bit better secure random number generation
import math

E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b
print("Base point:\n", P)
print("p :", p)
print("a :", a)
print("b :", b)
print("n :", n)

k = Random.new().read(int(math.log(n,2)))
k = int.from_bytes(k, byteorder='big')%n

Q = k*P
print("\nQ:\n", Q)
#print("Q on curve?", E.is_on_curve(Q))

In [ ]:
##################################    CLIENT.PY    ##################################

# !pip install pycryptodome

import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

API_URL = 'http://harpoon1.sabanciuniv.edu:9999'

stuID = 27851

#Server's Identitiy public key
IKey_Ser_x_hex = 0x1d42d0b0e55ccba0dd86df9f32f44c4efd7cbcdbbb7f36fd38b2ca680ab126e9
IKey_Ser_y_hex = 0xce091928fa3738dc18f529bf269ade830eeb78672244fd2bdfbadcb26c4894ff

IKey_Ser_x_dec = 13235124847535533099468356850397783155412919701096209585248805345836420638441
IKey_Ser_y_dec = 93192522080143207888898588123297137412359674872998361245305696362578896786687

##################################    IK FUNCTIONS (FIRST KEY TYPE)    ##################################

def PrivatePublicKeyGenerator():
  #print("\nKey generation process started")
  secret = random.randint(1, n - 2)
  #print("Randomly generated secret (sA or private key): ", secret)
  public_key = secret * P
  #print("\nKey generation completed, public key x component:", public_key.x, "public key y component:", public_key.y)

  return public_key, secret

#IK private and public created here to make code consistent with given IKRegVerify()
IKey_Pub, IKey_Pr = PrivatePublicKeyGenerator()
IKey_Pub_x = IKey_Pub.x
IKey_Pub_y = IKey_Pub.y

def IKSignatureGenerator(secret, message):
  #print("\nIK signing process started")
  message_to_sign = message.to_bytes((message.bit_length() + 7) // 8, byteorder='big')
  kRandom = 1363173 # random.randint(1, n - 2)
  print("In signature generation I fixed the random variable to 1363173 so that you can re-generate if you want\n")
  #print("Randomly generated k: ", kRandom)
  uppercase_r = kRandom * P
  #print("R value: ", uppercase_r)
  lowercase_r = uppercase_r.x % n
  r_byte_version = lowercase_r.to_bytes((lowercase_r.bit_length() + 7) // 8, byteorder='big')
  #print("r value: ", lowercase_r)
  binary_digest_hash = SHA3_256.new(r_byte_version + message_to_sign).digest()
  int_converted_hash = int.from_bytes(binary_digest_hash, 'big')
  h = int_converted_hash % n
  #print("h value: ", h)
  s = (kRandom - (secret * h)) % n
  #print("s value: ", s)
  #print("\nIK signature generation completed.")

  return h, s

def SignatureVerifier(h, s, QA, message):
  #print("\nSign verification started")
  message_to_sign = message.to_bytes((message.bit_length() + 7) // 8, byteorder='big')
  uppercase_v = s * P + h * QA
  lowercase_v = uppercase_v.x % n
  lowercase_v_byte_version = lowercase_v.to_bytes((lowercase_v.bit_length() + 7) // 8, byteorder='big')
  #print("V value: ", uppercase_v, "v value: ", lowercase_v)
  binary_digest_hash = SHA3_256.new(lowercase_v_byte_version + message_to_sign).digest()
  int_converted_hash = int.from_bytes(binary_digest_hash, 'big')
  quote_h = int_converted_hash % n
  #print("h' value: ", quote_h)
  if(quote_h != h):
    print("Verification failed!")
  else:
    print("Verified succesfully!")

  #print("\nSign verification completed")

##################################    IK FUNCTIONS DONE  ##################################

##################################    SPK FUNCTIONS (SECOND KEY TYPE)    ##################################

#Generator is same as IK generator
def SPKGenerator():
  print("\nSPK Public key generation started")
  secret = random.randint(1, n - 2)
  print("Randomly generated secret (sA): ", secret)
  public_key = secret * P
  print("\nSPK Public key generation completed, x component:", public_key.x, "y component:", public_key.y)

  return public_key, secret

#Only message parsing is different from IK
def SPKSignatureGenerator(secret, messageObj):
  print("\nSPK  signing started")
  messageObj_x_part = messageObj.x.to_bytes((messageObj.x.bit_length() + 7) // 8, byteorder='big')
  messageObj_y_part = messageObj.y.to_bytes((messageObj.y.bit_length() + 7) // 8, byteorder='big')
  kRandom = random.randint(1, n - 2)
  #print("Randomly generated k: ", kRandom)
  uppercase_r = kRandom * P
  #print("R value: ", uppercase_r)
  lowercase_r = uppercase_r.x % n
  r_byte_version = lowercase_r.to_bytes((lowercase_r.bit_length() + 7) // 8, byteorder='big')
  #print("r value: ", lowercase_r)
  binary_digest_hash = SHA3_256.new(r_byte_version + (messageObj_x_part + messageObj_y_part)).digest()
  int_converted_hash = int.from_bytes(binary_digest_hash, 'big')
  h = int_converted_hash % n
  #print("h value: ", h)
  s = (kRandom - (secret * h)) % n
  #print("s value: ", s)
  print("\nSPK signature generation completed.")

  return h, s

# Verifier is same as IK

##################################    SPK FUNCTIONS DONE  ##################################

##################################    HMAC FUNCTIONS  ##################################

def HMACGenerator(spkSecret):
  server_IK_key = Point(IKey_Ser_x_dec, IKey_Ser_y_dec, E)
  T = spkSecret * server_IK_key
  #print("T value:", T)
  givenMessage = b'TheHMACKeyToSuccess'
  U = givenMessage + T.y.to_bytes((T.y.bit_length() + 7) // 8, byteorder='big') + T.x.to_bytes((T.x.bit_length() + 7) // 8, byteorder='big')
  binary_digest_HMACValue = SHA3_256.new(U).digest()
  print("HMAC value:", binary_digest_HMACValue)
  int_converted_hash = int.from_bytes(binary_digest_HMACValue, 'big')
  #print("HMAC value:", int_converted_hash)

  return int_converted_hash

##################################    HMAC FUNCTIONS DONE  ##################################

##################################    OTK FUNCTIONS  ##################################

def OTKGenerator(HMAC_Value):
  print("Creating OTKs starting from index 0...")
  HMAC_Value_bytewise = HMAC_Value.to_bytes((HMAC_Value.bit_length() + 7) // 8, byteorder='big')
  otk_list = []

  for otkId in range(10):
    print("\nCurrently generating OTK key pair with id", otkId)
    publicOTKKey, privateOTKKey  = PrivatePublicKeyGenerator()
    #print("Current public key:", publicOTKKey, "Current private key:", privateOTKKey)
    concatedPublicKey = publicOTKKey.x.to_bytes((publicOTKKey.x.bit_length() + 7) // 8, byteorder='big') + publicOTKKey.y.to_bytes((publicOTKKey.y.bit_length() + 7) // 8, byteorder='big')
    #print("Concatenated public key to sign:", concatedPublicKey)
    #Should be converted to hexdigit given sample run!
    current_hmac_obj_hexdigit = HMAC.new(HMAC_Value_bytewise, concatedPublicKey, SHA256).hexdigest()
    #print("Current hmac hexdigit value:", current_hmac_obj_hexdigit)
    OTKReg(otkId, publicOTKKey.x, publicOTKKey.y, current_hmac_obj_hexdigit)
    otk_list.append( [publicOTKKey.x, publicOTKKey.y, privateOTKKey] )

  print("\nSuccesfully all OTKS are generated, here is list:")
  print(otk_list)

  return otk_list

otkGeneratedID = 9

def extraOTKGenerator(HMAC_Value):
  HMAC_Value_bytewise = HMAC_Value.to_bytes((HMAC_Value.bit_length() + 7) // 8, byteorder='big')
  otk_list = []

  newID = otkGeneratedID + 1
  print("\nCurrently generating OTK key pair with id", newID)
  publicOTKKey, privateOTKKey  = PrivatePublicKeyGenerator()
  #print("Current public key:", publicOTKKey, "Current private key:", privateOTKKey)
  concatedPublicKey = publicOTKKey.x.to_bytes((publicOTKKey.x.bit_length() + 7) // 8, byteorder='big') + publicOTKKey.y.to_bytes((publicOTKKey.y.bit_length() + 7) // 8, byteorder='big')
  #print("Concatenated public key to sign:", concatedPublicKey)
  #Should be converted to hexdigit given sample run!
  current_hmac_obj_hexdigit = HMAC.new(HMAC_Value_bytewise, concatedPublicKey, SHA256).hexdigest()
  #print("Current hmac hexdigit value:", current_hmac_obj_hexdigit)
  OTKReg(newID, publicOTKKey.x, publicOTKKey.y, current_hmac_obj_hexdigit)
  otk_list.append( [publicOTKKey.x, publicOTKKey.y, privateOTKKey] )

  print("\nSuccesfully extra OTKS are generated")

  return otk_list

##################################    OTK FUNCTIONS DONE  ##################################

def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)
    print(response.json())

def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    else:
        print(response.json())
        f = open('Identity_Key.txt', 'w')
        f.write("IK.Prv: "+str(IKey_Pr)+"\n"+"IK.Pub.x: "+str(IKey_Pub_x)+"\n"+"IK.Pub.y: "+str(IKey_Pub_y))
        f.close()

def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)
    print(response.json())

def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)
    if((response.ok) == False): print(response.json())

#IK PART

#h, s = IKSignatureGenerator(IKey_Pr, stuID)
#SignatureVerifier(h, s, IKey_Pr*P, stuID)
#IKRegReq(h, s, IKey_Pub.x, IKey_Pub.y)
#mailVerificationCode = input("Please enter the verification code sent to the mail: ")
#IKRegVerify(int(mailVerificationCode))

#IK DONE

#SPK PART

#SPK_public, SPK_secret = SPKGenerator()
#h, s = SPKSignatureGenerator(IKey_Pr, SPK_public)

# VERIFY SPK
#spkpubx = SPK_public.x
#spkpuby = SPK_public.y
#messageObj_x_part = spkpubx.to_bytes((spkpubx.bit_length() + 7) // 8, byteorder='big')
#messageObj_y_part = spkpuby.to_bytes((spkpuby.bit_length() + 7) // 8, byteorder='big')
#SignatureVerifier(h, s, IKey_Pr*P, int.from_bytes(messageObj_x_part + messageObj_y_part, 'big'))
# SPK VERIFICATION DONE
#SPKReg(h, s, spkpubx, spkpuby)

# SPK DONE

#HmacValue = HMACGenerator(SPK_secret)
#otk_arr = OTKGenerator(HmacValue)

# RESET CODES

#hResetSPK, sResetSPK = IKSignatureGenerator(IKey_Pr, stuID)
#ResetSPK(hResetSPK, sResetSPK)
#ResetOTK(hResetSPK, sResetSPK)
ResetIK(111111)

# RESET CODES DONE

print("Here is my private Identity Key")
print(IKey_Pr)
print("I've already registered my IK to server. So I'm skipping IK registration part")
print("My ID number is 11111")
print("Signature of my ID number is:")
ID_h, ID_s = IKSignatureGenerator(IKey_Pr, stuID)
print("h =", ID_h)
print("s =", ID_s)

print("Sending signature and my IKEY to server via IKRegReq() function in json format")
IKRegReq(ID_h, ID_s, IKey_Pub.x, IKey_Pub.y)
mailVerificationCode = input("Please enter the verification code sent to the mail: ")
print("Sending the verification code to server via IKRegVerify() function in json format")
IKRegVerify(int(mailVerificationCode))

SPK_public, SPK_secret = SPKGenerator()
h, s = SPKSignatureGenerator(IKey_Pr, SPK_public)
SPKReg(h, s, SPK_public.x, SPK_public.y)

HmacValue = HMACGenerator(SPK_secret)
otk_arr = OTKGenerator(HmacValue)

In [ ]:
##################################    PHASE 2  ##################################

#Pseudo-client will send you 5 messages to your inbox via server when you call this function
def PseudoSendMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsg"), json = mes)
    print(response.json())

#Get your messages. server will send 1 message from your inbox
def ReqMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
    print(response.json())
    if((response.ok) == True):
        res = response.json()
        return res["IDB"], res["OTKID"], res["MSGID"], res["MSG"], res["IK.X"], res["IK.Y"], res["EK.X"], res["EK.Y"]

#Get the list of the deleted messages' ids.
def ReqDelMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqDelMsgs"), json = mes)
    print(response.json())
    if((response.ok) == True):
        res = response.json()
        return res["MSGID"]

#If you decrypted the message, send back the plaintext for checking
def Checker(stuID, stuIDB, msgID, decmsg):
    mes = {'IDA':stuID, 'IDB':stuIDB, 'MSGID': msgID, 'DECMSG': decmsg}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
    print(response.json())

def DecrpytMessages(h, s):
  decryptedMessages = []
  kdfNext = -1
  for messageIdx in range(5):
    current_msg = ReqMsg(h, s)
    stuIDB = current_msg[0]
    currOTKId = current_msg[1]
    messageId = current_msg[2]
    message = current_msg[3]
    print("I got this from client", stuIDB, ":")
    print(message)
    print("\nConverting message to bytes to decrypt it...")
    message_byte_version = message.to_bytes((message.bit_length() + 7) // 8, byteorder='big')
    print("Converted message is:", message_byte_version)
    IKX = current_msg[4]
    IKY = current_msg[5]
    EKX = current_msg[6]
    EKY = current_msg[7]
    print("\nGenerating the key Ks, Kenc, & Khmac and then the HMAC value ..")
    ik_point = Point(IKX, IKY, E)
    ek_point = Point(EKX, EKY, E)

    T_1 = ik_point * SPK_secret
    T_1_x = T_1.x.to_bytes((T_1.x.bit_length() + 7) // 8, byteorder='big')
    T_1_y = T_1.y.to_bytes((T_1.y.bit_length() + 7) // 8, byteorder='big')
    T_2 = ek_point * IKey_Pr
    T_2_x = T_2.x.to_bytes((T_2.x.bit_length() + 7) // 8, byteorder='big')
    T_2_y = T_2.y.to_bytes((T_2.y.bit_length() + 7) // 8, byteorder='big')
    T_3 = ek_point * SPK_secret
    T_3_x = T_3.x.to_bytes((T_3.x.bit_length() + 7) // 8, byteorder='big')
    T_3_y = T_3.y.to_bytes((T_3.y.bit_length() + 7) // 8, byteorder='big')
    T_4 = ek_point * otk_arr[currOTKId][2]
    T_4_x = T_4.x.to_bytes((T_4.x.bit_length() + 7) // 8, byteorder='big')
    T_4_y = T_4.y.to_bytes((T_4.y.bit_length() + 7) // 8, byteorder='big')
    U = T_1_x + T_1_y + T_2_x + T_2_y + T_3_x + T_3_y + T_4_x + T_4_y + b'WhatsUpDoc'

    K_s_byte = SHA3_256.new(U).digest()
    K_s_int = int.from_bytes(K_s_byte, 'big')
    if(messageId == 1):
      kdfNext = K_s_byte

    message_nonce_part = message_byte_version[:8]
    #print("Nonce", message_nonce_part)
    message_hmac_part = message_byte_version[-32:]
    #print("HMAC", message_hmac_part)
    message_ciphertext_part = message_byte_version[8:-32]
    #print("CIPHERTEXT", message_ciphertext_part)


    print("hmac is: ", message_hmac_part)

    kENC = SHA3_256.new(kdfNext+b'JustKeepSwimming').digest()
    kHMAC = SHA3_256.new(kdfNext+kENC+b'HakunaMatata').digest()
    kdfNext = SHA3_256.new(kENC+kHMAC+b'OhanaMeansFamily').digest()

    hmac_to_verify = HMAC.new(kHMAC, message_ciphertext_part, SHA256).digest()

    print(hmac_to_verify)

    if(hmac_to_verify != message_hmac_part):
      print("Hmac value couldn't be verified")
      Checker(stuID, stuIDB, messageId, "INVALIDHMAC")

    else:
      decrypted_plaintext = AES.new(key = kENC, mode = AES.MODE_CTR, nonce = message_nonce_part).decrypt(message_ciphertext_part)
      print("Hmac value is verified")
      print("The collected plaintext: ", decrypted_plaintext)
      decoded_plaintext = decrypted_plaintext.decode("UTF-8")
      decryptedMessages.append( (messageId, decoded_plaintext) )
      Checker(stuID, stuIDB, messageId, decoded_plaintext)

    print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

  return decryptedMessages

def decryptVerifierForEncoding(message_int):
    client_message_bytes = message_int.to_bytes((message_int.bit_length() + 7) // 8, byteorder='big')
    decrypted_plaintext = AES.new(key = kENC, mode = AES.MODE_CTR, nonce = client_message_bytes[:8]).decrypt(client_message_bytes[8:-32])
    print("Hmac value is verified")
    print("The collected plaintext: ", decrypted_plaintext)
    decoded_plaintext = decrypted_plaintext.decode("UTF-8")

    print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")


print("\nChecking the inbox for incoming messages")
print("+++++++++++++++++++++++++++++++++++++++++++++")

ID_h, ID_s = IKSignatureGenerator(IKey_Pr, stuID)
print("Signing my stuID with my private IK\n")

print("In signature generation I fixed the random variable to 1363173 so that you can re-generate if you want\n")
PseudoSendMsg(ID_h,ID_s)

print("Starting to decrypt messages")
print("+++++++++++++++++++++++++++++++++++++++++++++\n")

decryptedMessagesArr = DecrpytMessages(ID_h, ID_s)

print("\nChecking whether there were some deleted messages!!")
print("==========================================\n")


deletedMessagesArr = ReqDelMsg(ID_h, ID_s)

for currMessageId in range(1, 6):
  if currMessageId in deletedMessagesArr:
    print("Message", currMessageId, "-", "Was deleted by the sender", "X")
    continue
  if decryptedMessagesArr is not None:
    for messageId, message in decryptedMessagesArr:
      if currMessageId == messageId:
        print("Message", currMessageId, "-", message, "-", "Read")

In [ ]:
##################################    PHASE 3  ##################################

import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

API_URL = 'http://harpoon1.sabanciuniv.edu:9999/'

stuID = 11111 # Enter your student ID
stuIDB = 11111

############## The new functions of phase 3 ###############

#Pseudo-client will send you 5 messages to your inbox via server when you call this function
def PseudoSendMsgPH3(h,s):
    mes = {'ID': stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsgPH3"), json=mes)
    print(response.json())

# Send a message to client idB
def SendMsg(idA, idB, otkID, msgid, msg, ikx, iky, ekx, eky):
    mes = {"IDA": idA, "IDB": idB, "OTKID": int(otkID), "MSGID": msgid, "MSG": msg, "IK.X": ikx, "IK.Y": iky, "EK.X": ekx, "EK.Y": eky}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SendMSG"), json=mes)
    print(response.json())


# Receive KeyBundle of the client stuIDB
def reqKeyBundle(stuID, stuIDB, h, s):
    key_bundle_msg = {'IDA': stuID, 'IDB':stuIDB, 'S': s, 'H': h}
    print("Requesting party B's Key Bundle ...")
    response = requests.get('{}/{}'.format(API_URL, "ReqKeyBundle"), json=key_bundle_msg)
    print(response.json())
    if((response.ok) == True):
        print(response.json())
        res = response.json()
        return res['KEYID'], res['IK.X'], res['IK.Y'], res['SPK.X'], res['SPK.Y'], res['SPK.H'], res['SPK.s'], res['OTK.X'], res['OTK.Y']

    else:
        return -1, 0, 0, 0, 0, 0, 0, 0, 0


#Status control. Returns #of messages and remained OTKs
def Status(stuID, h, s):
    mes = {'ID': stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "Status"), json=mes)
    print(response.json())
    if (response.ok == True):
        res = response.json()
        return res['numMSG'], res['numOTK'], res['StatusMSG']


############## The new functions of BONUS ###############

# Exchange partial keys with users 2 and 4
def ExchangePartialKeys(stuID, z1x, z1y, h, s):
    request_msg = {'ID': stuID, 'z1.x': z1x, 'z1.y': z1y, 'H': h, 'S': s}
    print("Sending your PK (z) and receiving others ...")
    response = requests.get('{}/{}'.format(API_URL, "ExchangePartialKeys"), json=request_msg)
    if ((response.ok) == True):
        print(response.json())
        res = response.json()
        return res['z2.x'], res['z2.y'], res['z4.x'], res['z4.y']
    else:
        print(response.json())
        return 0, 0, 0, 0


# Exchange partial keys with user 3
def ExchangeXs(stuID, x1x, x1y, h, s):
    request_msg = {'ID': stuID, 'x1.x': x1x, 'x1.y': x1y, 'H': h, 'S': s}
    print("Sending your x and receiving others ...")
    response = requests.get('{}/{}'.format(API_URL, "ExchangeXs"), json=request_msg)
    if ((response.ok) == True):
        print(response.json())
        res = response.json()
        return res['x2.x'], res['x2.y'], res['x3.x'], res['x3.y'], res['x4.x'], res['x4.y']
    else:
        print(response.json())
        return 0, 0, 0, 0, 0, 0

# Check if your conference key is correct
def BonusChecker(stuID, Kx, Ky):
    mes = {'ID': stuID, 'K.x': Kx, 'K.y': Ky}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "BonusChecker"), json=mes)
    print(response.json())


# Sender Function
def SendEncryptedMessages(receiverBundle, decryptedMessagesArr, deletedMessagesArr):
  print(decryptedMessagesArr)
  randNumberEK = random.randint(1, n-2)
  sender_EK_point = randNumberEK * P

  kdfNext = -1
  for currMessageId in range(1, 6):
    if currMessageId in deletedMessagesArr:
      print("Message with ID", currMessageId, "is deleted and it will not be sent to pseudoclient!")
      print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

      continue

    for messageId, message in decryptedMessagesArr:
      if currMessageId == messageId:
        print("Get a message from the inbox ...")
        print("Generating the KDF chain for the encryption and the MAC value generation")
        print("Generating session key / Phase 3...")

        message_byte_version = message.encode('utf-8')

        otk_id_receiver = receiverBundle[0]
        ik_x_receiver = receiverBundle[1]
        ik_y_receiver = receiverBundle[2]
        spk_x_receiver = receiverBundle[3]
        spk_y_receiver = receiverBundle[4]
        otk_x_receiver = receiverBundle[7]
        otk_y_receiver = receiverBundle[8]

        receiver_SPK_point = Point(spk_x_receiver,spk_y_receiver, E)
        receiver_IK_point = Point(ik_x_receiver,ik_y_receiver, E)
        receiver_OTK_point = Point(otk_x_receiver,otk_y_receiver, E)

        T_1 = IKey_Pr * receiver_SPK_point
        T_1_x = T_1.x.to_bytes((T_1.x.bit_length() + 7) // 8, byteorder='big')
        T_1_y = T_1.y.to_bytes((T_1.y.bit_length() + 7) // 8, byteorder='big')
        T_2 = randNumberEK * receiver_IK_point
        T_2_x = T_2.x.to_bytes((T_2.x.bit_length() + 7) // 8, byteorder='big')
        T_2_y = T_2.y.to_bytes((T_2.y.bit_length() + 7) // 8, byteorder='big')
        T_3 = randNumberEK * receiver_SPK_point
        T_3_x = T_3.x.to_bytes((T_3.x.bit_length() + 7) // 8, byteorder='big')
        T_3_y = T_3.y.to_bytes((T_3.y.bit_length() + 7) // 8, byteorder='big')
        T_4 = randNumberEK * receiver_OTK_point
        T_4_x = T_4.x.to_bytes((T_4.x.bit_length() + 7) // 8, byteorder='big')
        T_4_y = T_4.y.to_bytes((T_4.y.bit_length() + 7) // 8, byteorder='big')
        U = T_1_x + T_1_y + T_2_x + T_2_y + T_3_x + T_3_y + T_4_x + T_4_y + b'WhatsUpDoc'

        print("U is:")
        print(U)

        K_s_byte = SHA3_256.new(U).digest()
        if(kdfNext == -1):
          kdfNext = K_s_byte

        kENC = SHA3_256.new(kdfNext+b'JustKeepSwimming').digest()
        kHMAC = SHA3_256.new(kdfNext+kENC+b'HakunaMatata').digest()
        kdfNext = SHA3_256.new(kENC+kHMAC+b'OhanaMeansFamily').digest()
        AESObj = AES.new(kENC, AES.MODE_CTR)
        encryptedMessage = AESObj.encrypt(message_byte_version)
        hmac_to_verify = HMAC.new(kHMAC, encryptedMessage, SHA256).digest()
        message_to_send_int = int.from_bytes(AESObj.nonce + encryptedMessage + hmac_to_verify, byteorder="big")

        client_message_bytes = message_to_send_int.to_bytes((message_to_send_int.bit_length() + 7) // 8, byteorder='big')
        decrypted_plaintext = AES.new(key = kENC, mode = AES.MODE_CTR, nonce = client_message_bytes[:8]).decrypt(client_message_bytes[8:-32])
        print("Hmac value is verified")
        print("The collected plaintext: ", decrypted_plaintext)
        decoded_plaintext = decrypted_plaintext.decode("UTF-8")

        print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

        print("Sending the message to the server, so it would deliver it to pseudo-client/user whenever it is active ...")

        SendMsg(stuID, stuIDB, otk_id_receiver, currMessageId, message_to_send_int, IKey_Pub.x, IKey_Pub.y, sender_EK_point.x, sender_EK_point.y)

        print("\n+++++++++++++++++++++++++++++++++++++++++++++\n")

print("Signing The stuIDB of party B with my private IK")
h_stuIDB, s_stuIDB = IKSignatureGenerator(IKey_Pr, stuIDB)

receiver_bundle = reqKeyBundle(stuID, stuIDB, h_stuIDB, s_stuIDB)

print("Verifying the server's SPK...")
spkpubx = receiver_bundle[3]
spkpuby = receiver_bundle[4]
spk_h = receiver_bundle[5]
spk_s = receiver_bundle[6]
messageObj_x_part = spkpubx.to_bytes((spkpubx.bit_length() + 7) // 8, byteorder='big')
messageObj_y_part = spkpuby.to_bytes((spkpuby.bit_length() + 7) // 8, byteorder='big')

SignatureVerifier(spk_h, spk_s, 75151745874135875874574268787*P, int.from_bytes(messageObj_x_part + messageObj_y_part, 'big'))

SendEncryptedMessages(receiver_bundle, decryptedMessagesArr, deletedMessagesArr)

print("Checking status of remaining OTKs after sending 5 messages got from client")

statusRes = Status(stuID, ID_h, ID_s)
remainingOTK = statusRes[1]
extraOTKNeeded = 10 - remainingOTK

if remainingOTK < 10:
  print("OTKs are not enough for 10 message blocks currently creating", extraOTKNeeded, "more OTKs")
  for i in range(extraOTKNeeded):
    extraOTKGenerator(HmacValue)
    otkGeneratedID += 1

# Reset Part

hResetSPK, sResetSPK = IKSignatureGenerator(IKey_Pr, stuID)
ResetSPK(hResetSPK, sResetSPK)
ResetOTK(hResetSPK, sResetSPK)
mailResetCode = input("Please enter the IK reset code sent to the mail: ")
ResetIK(mailResetCode)